In [116]:
import os
# on se replace à la racine du projet pour utiliser les modules créé dans le projet
os.chdir("../../..")
from data_storage.sql_alchemy_connect.alchemy_cur import AlchemyConn
import pandas as pd



In [117]:
connector = AlchemyConn()
df = pd.read_sql('SELECT * FROM raw_data',con=connector.get_engine())

In [118]:
print(df.head())
print(df.shape)

   id  surface_reelle_bati   prix_m2  nombre_pieces_principales  longitude  \
0   3                 78.0  11643.60                        3.0    2.25112   
1   4                 70.0   8571.43                        3.0    2.25190   
2   5                 77.0  10066.60                        4.0    2.25243   
3   6                 67.0   8208.96                        3.0    2.25287   
4   7                 74.0   9331.08                        3.0    2.25137   

   latitude  nombre_lots  annee  arrondissement  distance_datashop_km  ...  \
0   48.8430            2   2020              16              1.123880  ...   
1   48.8384            4   2020              16              0.152864  ...   
2   48.8377            1   2020              16              0.769220  ...   
3   48.8374            2   2020              16              0.410568  ...   
4   48.8393            2   2020              16              0.269765  ...   

   distance_universite_km  distance_ecole_km  distance_metro_k

In [119]:
y = df['prix_m2']
X = df.drop(['prix_m2'],axis=1)
print(y.head())

0    11643.60
1     8571.43
2    10066.60
3     8208.96
4     9331.08
Name: prix_m2, dtype: float64


In [120]:
print(X.isnull().sum())
print('\n-------\n')
print(y.isnull().sum())

id                           0
surface_reelle_bati          0
nombre_pieces_principales    0
longitude                    0
latitude                     0
nombre_lots                  0
annee                        0
arrondissement               0
distance_datashop_km         0
distance_espace_vert_km      0
distance_college_km          0
distance_universite_km       0
distance_ecole_km            0
distance_metro_km            0
distance_TER_km              0
distance_POI_min_km          0
proche_POI_1km               0
nb_POIs_inf_1km              0
cle_interop_adr_proche       0
distance_batiment_m          0
annee_construction_dpe       0
dtype: int64

-------

0


In [121]:
X.dtypes

id                             int64
surface_reelle_bati          float64
nombre_pieces_principales    float64
longitude                    float64
latitude                     float64
nombre_lots                    int64
annee                          int64
arrondissement                 int64
distance_datashop_km         float64
distance_espace_vert_km      float64
distance_college_km          float64
distance_universite_km       float64
distance_ecole_km            float64
distance_metro_km            float64
distance_TER_km              float64
distance_POI_min_km          float64
proche_POI_1km                 int64
nb_POIs_inf_1km                int64
cle_interop_adr_proche        object
distance_batiment_m          float64
annee_construction_dpe       float64
dtype: object

### Les arrondissements sont difficilement OneHotEncoding

In [122]:
import plotly.express as px

X.arrondissement.value_counts()
fig = px.bar(x=X.arrondissement.unique(),y=X.arrondissement.value_counts())
fig.show()


Il serait intéressant de faire un premier clustering pour créer des catégories et faire diminuer le nombre de zones

In [123]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

cluster_df = df[['longitude','latitude','arrondissement']]
print(cluster_df.dtypes)
kmeans = KMeans(n_clusters=5, random_state=0)

sc = StandardScaler()
X = sc.fit_transform(cluster_df)

kmeans.fit(X)
inverse = sc.inverse_transform(kmeans.cluster_centers_)
mapp = pd.DataFrame(inverse, columns=['longitude','latitude','arrondissement']).round(2)
print(mapp.shape)
maps = px.scatter_map(mapp,lat="latitude",
    lon="longitude",
    size=mapp.arrondissement.value_counts(),        # Taille des bulles
    hover_name="arrondissement",    # Label au survol
    zoom=11,
    center={"lat": 48.8566, "lon": 2.3522},
    map_style="carto-positron"  # style gratuit
    )
maps.show()



longitude         float64
latitude          float64
arrondissement      int64
dtype: object
(5, 3)


In [124]:
wcss =  []
k = []
for i in range (1,11):
    kmeans2 = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans2.fit(X)
    wcss.append(kmeans2.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))


WCSS for K=1 --> 413751.00000000064
WCSS for K=2 --> 270724.61929278355
WCSS for K=3 --> 166027.20661572347
WCSS for K=4 --> 129181.88072487153
WCSS for K=5 --> 89613.60265983547
WCSS for K=6 --> 69950.67656685977
WCSS for K=7 --> 63788.24199963034
WCSS for K=8 --> 45943.19780174204
WCSS for K=9 --> 40421.02754270513
WCSS for K=10 --> 36358.434495751375


On selectionne 5 clusters car moins de décroissance entre 5 et 6

Je teste maintenant la prédiction

In [138]:
second_pred = cluster_df.sample()
print(second_pred)
second_pred = sc.transform(second_pred)
results2 = kmeans.predict(second_pred)
print(f"L'appartement appartient au groupe : {results2[0]}")


       longitude  latitude  arrondissement
69011    2.33724   48.8858              18
L'appartement appartient au groupe : 3


Je veux maintenant affecter un numéro de groupe à chacune des lignes des données 

In [ ]:
def predictGroupeKmeans():
    
    return